# Decorators, how do they work??

Understanding how decorators work _really_ helps me understand how, why, and when to use them, and enables me to write my own

#### Spoiler: They're **not** magic

#### Well, maybe just a little bit...

#### Nope, just some syntactic sugar, not really magic.

**Syntactic sugar**: special language features to make an already available pattern easier to write and/or clearer to read for humans)

#### They are _**VERY**_ abstract, though...

## Callables "lightning talk" refresher

### What are "callables"?  Why make up words?

- "A callable is anything that can be called" -- [top StackOverflow answer][1]; true, but not very helpful
- Why "callable"?  Too many distinct kinds; the term "function" has baggage
[1]: https://stackoverflow.com/questions/111234/what-is-a-callable-in-python#111255

### "Snippets of code"

- Functions, lambdas
- Easy to take for granted, but very powerful:
    - Can operate on data - take parameters, return results
    - Callables can call other callables

### "Code + data"

- Classes (`__init__`), callable objects, bound methods, ...

### "Code _<u>as</u>_ data"

- callbacks, helpers (e.g. `sort`'s `key=`)
- closures (aka "inner functions")

### Callables can take callables...

In [297]:
def hello(printer):
    printer("Hello world")

hello(print)

Hello world


In [298]:
def exclamation(s):
    print(f"{s}!")

hello(exclamation)

Hello world!


## ...and return callables!

In [253]:
def greeter(printer):
    def greet(name):
        # Inner function has - and KEEPS - the context in 
        # which is was created ("closes over", aka "closure")
        printer(f"Hello {name}")
    return greet

hi = greeter(print)
hi_bang = greeter(exclamation)

hi("Python Study Group")
hi_bang("Python Study Group")

Hello Python Study Group
Hello Python Study Group!


## Let's try this again

In [309]:
from random import choice, random
def flaky():
    return choice(["Okay", None])
    
flaky()

'Okay'

In [261]:
def retry(f):
    tries = 5
    for t in range(tries):
        result = f()
        if result:
            return result
        print(f"fail {t}")
    return result

retry(flaky)

fail 0
fail 1


'Okay'

In [328]:
def retried(f):
    tries = 5
    def retry_wrapper():
        for t in range(tries):
            result = f()
            if result:
                return result
            print(f"fail {t}")
        return result
    return retry_wrapper

flaky = retried(flaky)
flaky()

'Okay'

## Add a little syntactic sugar...

In [167]:
@retried
def flaky():
    return choice(["Okay", None])

flaky()

fail 0
fail 1
fail 2
fail 3


'Okay'

## How does that work?

**retried**: expression that _<u>takes</u>_ a callable and _<u>returns</u>_ a callable that _<u>replaces<u>_ the following one

In [179]:
retried

<function __main__.retried(f)>

**@**: the sugar, automatically calls the decorator _<u>at declaration time</u>_ with the following one and does the replacement, basically like we did above.

## Getting fancy

Let's make a version with configurable tries, say, `retry_n`...

...but wait, if the `@` automatically calls it with the following thing - _<u>just</u>_ the following thing - where would arguments for _<u>it</u>_ go?

## Inception

### Callables can return _<u>generators!</u>_

In [299]:
def retry_n(tries):
    def retried(f):
        def retry_wrapper():
            for t in range(tries):
                result = f()
                if result:
                    return result
                print(f"fail {t}")
            return result
        return retry_wrapper
    return retried

@retry_n(2)
def best_effort():
    return choice(["Okay", None])

In [308]:
best_effort()

fail 0
fail 1


### What's going on here?
```python
def best_effort():
    return choice(["Okay", None])
best_effort = retry_n(2)(best_effort)
```

## Multiple decorators

In [283]:
def bold(f):
    def stars():
        return(f"*{f()}*")
    return stars

def italic(f):
    def unders():
        return(f"_{f()}_")
    return unders

In [289]:
@bold
@italic
def hi():
    return "Hello"

hi()
# Will it be '_*Hello*_' or '*_Hello_*'?

'*_Hello_*'

```python
hi = bold(italics(hi))
```

In [291]:
@italics
@bold
def hi():
    return "Hello"

hi()

'_*Hello*_'

```python
hi = italics(bold(hi))
```

## Professional touches

In [332]:
@retried
def flaky_greet(name, reliablility=0.5):
    return f"Hello {name}" if random() <= reliablility else None

flaky_greet("Ben")

TypeError: retry_wrapper() takes 0 positional arguments but 1 was given

In [316]:
import functools
def retried(f):
    tries = 5
    @functools.wraps(f)
    def retry_wrapper(*args, **kwargs):
        for t in range(tries):
            result = f(*args, **kwargs)
            if result:
                return result
            print(f"fail {t}")
        return result
    return retry_wrapper

In [324]:
@retried
def flaky_greet(name, reliablility=0.5):
    return f"Hello {name}" if random() <= reliablility else None

In [325]:
flaky_greet()

TypeError: flaky_greet() missing 1 required positional argument: 'name'

In [326]:
flaky_greet("Ben")

fail 0


'Hello Ben'

In [327]:
flaky_greet("Ben", reliablility=0.95)

'Hello Ben'

## Uses for decorators

- Change the behavior of a callable (e.g. retry, lru_cache, ddt)
- Change the behavior of _something else_ "during" a callable
- "Registering" callables
    - Often returns original
- Implementation swap
    - Avoid re-checking invariant condition
- ...and more!

In [333]:
from functools import lru_cache
import shutil
import subprocess as sub
import sys
def tput_lookup(*cap):
    "Get the value for a capname from tput, see man tput for more information"
    tp_cmd = ["tput",] + [ str(i) for i in cap ]
    result = sub.run(tp_cmd, stdout=sub.PIPE)
    r = ""  # Default to falsey
    if result.returncode == 0:  # If success, capability exists or property is true
        r = result.stdout.decode("utf-8")
        if r:
            try:
                r = int(r)  # Note: many of the int
            except ValueError:
                pass  # Nope, not an int
        else:
            r = True  # tput command success means "cap is true"
    return r

In [334]:
def tput_swap(dummy):
    "Decorator to replace a dummy implementation with calls to tput if appropreate and available"
    if sys.stdout.isatty() and shutil.which("tput"):
        return lru_cache()(tput_lookup)  # Note: some attributes (e.g. "cols") can change, so cache may be stale
    else:
        return dummy


@tput_swap
def tput(*cap):
    "Dummy tput implementation, just returns empty string for now"
    return ""  # This might not make much sense for some things (eg "cols"), so callers may need to test and use sensible defaults